In [1]:
# loading libraries
import os
import cv2  
import numpy as np

In [2]:
# defining the directory where images are located
image_directory = '/home/hduser/Desktop/BANANAS'

In [3]:
# defining the target image size
target_image_size = (128, 128)

In [4]:
# creating a function to load, resize, normalize, and assign labels to images
def load_resize_normalize_image(image_path, target_size):
    
    image = cv2.imread(os.path.join(image_directory, image_path + '.jpg'))

    # checking if the image was loaded successfully
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None, None

    # resize the image to the target size
    resized_image = cv2.resize(image, target_size)

    # normalising the image by dividing it by 255.0 to scale pixel values to [0, 1]
    normalized_image = resized_image / 255.0

    # assigning labels based on their names
    if image_path.startswith('g'):
        label = 0
    elif image_path.startswith('R'):
        label = 1
    elif image_path.startswith('Y'):
        label = 2
    else:
        label = 99

    return normalized_image, label

# creating lists to store ready images and labels
images = []
labels = []

# looping through image files and preprocess them
for image_filename in os.listdir(image_directory):
    if image_filename.endswith('.jpg'):
        image_path = os.path.splitext(image_filename)[0] 
        normalized_image, label = load_resize_normalize_image(image_path, target_image_size)
        
        if normalized_image is not None and label is not None:
            images.append(normalized_image)
            labels.append(label)

# converting lists to NumPy arrays for further processing
images = np.array(images)
labels = np.array(labels)

# printing the shapes of the loaded images and labels
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (297, 128, 128, 3)
Labels shape: (297,)


In [5]:
labels

array([2, 0, 1, 0, 1, 2, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 2, 2, 2,
       2, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1, 2,
       0, 2, 2, 0, 2, 0, 1, 1, 2, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0,
       2, 2, 2, 2, 1, 1, 0, 0, 2, 1, 1, 2, 1, 2, 0, 2, 0, 2, 0, 1, 2, 2,
       0, 0, 0, 1, 2, 0, 2, 0, 1, 1, 0, 2, 2, 1, 2, 0, 0, 1, 1, 2, 1, 2,
       0, 1, 0, 1, 1, 1, 2, 1, 2, 2, 2, 1, 0, 0, 0, 1, 2, 0, 2, 1, 1, 1,
       2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 2, 1, 1, 0, 1, 2, 2, 0, 2, 2, 1, 0,
       1, 2, 1, 2, 1, 0, 0, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 2, 0, 1, 0, 2, 1, 1, 2, 0, 2,
       0, 1, 1, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1, 1, 1, 2, 2, 2, 0,
       1, 0, 2, 1, 1, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 0, 2, 1, 2, 1, 0, 0,
       0, 2, 1, 0, 2, 2, 2, 0, 1, 2, 0, 1, 1, 0, 1, 2, 1, 1, 0, 0, 0, 2,
       2, 0, 1, 0, 0, 2, 1, 1, 2, 2, 0, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 1,
       2, 2, 0, 0, 2, 2, 1, 0, 2, 2, 0])

In [6]:
# splitting the data into 4 shards
num_shards = 4
shard_size = len(images) // num_shards

In [7]:
# creating lists to store shards images and labels
shards_images = []
shards_labels = []

for i in range(num_shards):
    start_idx = i * shard_size
    end_idx = (i + 1) * shard_size

    # extracting a shard of images and labels
    shard_images = images[start_idx:end_idx]
    shard_labels = labels[start_idx:end_idx]

    # appending the shard to the lists
    shards_images.append(shard_images)
    shards_labels.append(shard_labels)

# veryfing the sizes of the shards
for i, shard_images in enumerate(shards_images):
    print(f"Shard {i + 1}: Images shape: {shard_images.shape}, Labels shape: {shards_labels[i].shape}")

Shard 1: Images shape: (74, 128, 128, 3), Labels shape: (74,)
Shard 2: Images shape: (74, 128, 128, 3), Labels shape: (74,)
Shard 3: Images shape: (74, 128, 128, 3), Labels shape: (74,)
Shard 4: Images shape: (74, 128, 128, 3), Labels shape: (74,)


# CNN

In [8]:
# importing libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import multiprocessing

2023-09-20 22:22:59.582042: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 22:23:00.134233: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 22:23:00.135858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 22:23:01.946042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
# defining a CNN model
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax')) 
    return model


In [10]:
# defining a function to train and evaluate the model on a shard
def process_shard(shard_images, shard_labels):
    
    model = cnn_model()
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # training the model on the shard
    model.fit(shard_images, shard_labels, epochs=5, batch_size=32) 
    
    # fvaluating the model on the shard
    loss, accuracy = model.evaluate(shard_images, shard_labels)

    return loss, accuracy

In [11]:
# creating a multiprocessing pool with 2 CPUs
num_cpus = 2
with multiprocessing.Pool(processes=num_cpus) as pool:
    results = pool.starmap(process_shard, zip(shards_images[:2], shards_labels[:2]))


Epoch 1/5
Epoch 1/5
3/3 [==============================] - 1s 257ms/step - loss: 7.3671 - accuracy: 0.3378
Epoch 2/5
3/3 [==============================] - 1s 241ms/step - loss: 12.6860 - accuracy: 0.3514
Epoch 2/5
3/3 [==============================] - 1s 255ms/step - loss: 4.9240 - accuracy: 0.2973
Epoch 3/5
3/3 [==============================] - 1s 207ms/step - loss: 6.2121 - accuracy: 0.3919
Epoch 4/5
3/3 [==============================] - 1s 283ms/step - loss: 1.6462 - accuracy: 0.4730
Epoch 4/5
3/3 [==============================] - 1s 243ms/step - loss: 2.8240 - accuracy: 0.5541
Epoch 5/5
3/3 [==============================] - 1s 268ms/step - loss: 0.7735 - accuracy: 0.6216
Epoch 5/5
3/3 [==============================] - 0s 39ms/step - loss: 0.6483 - accuracy: 0.7432


In [12]:
 #create a multiprocessing pool with 2 CPUs - 4 shards
num_cpus = 2
with multiprocessing.Pool(processes=num_cpus) as pool:
    results = pool.starmap(process_shard, zip(shards_images[:4], shards_labels[:4]))


Epoch 1/5
Epoch 1/5
3/3 [==============================] - 1s 209ms/step - loss: 3.3105 - accuracy: 0.3784
Epoch 2/5
3/3 [==============================] - 2s 239ms/step - loss: 8.9735 - accuracy: 0.2568
Epoch 2/5
3/3 [==============================] - 1s 217ms/step - loss: 1.6269 - accuracy: 0.5946
Epoch 3/5
3/3 [==============================] - 1s 224ms/step - loss: 6.3792 - accuracy: 0.3108
Epoch 3/5
3/3 [==============================] - 1s 254ms/step - loss: 0.9695 - accuracy: 0.6757
Epoch 4/5
3/3 [==============================] - 1s 241ms/step - loss: 3.8264 - accuracy: 0.3108
Epoch 4/5
3/3 [==============================] - 1s 243ms/step - loss: 0.7040 - accuracy: 0.6892
Epoch 5/5
3/3 [==============================] - 1s 273ms/step - loss: 1.7789 - accuracy: 0.4324
Epoch 5/5
3/3 [==============================] - 0s 45ms/step - loss: 0.4580 - accuracy: 0.7973
Epoch 1/5
Epoch 1/5
3/3 [==============================] - 1s 195ms/step - loss: 7.0694 - accuracy: 0.4054
Epoch 2/5
3

In [13]:
# create a multiprocessing pool with 4 CPUs -4shards
num_cpus = 4
with multiprocessing.Pool(processes=num_cpus) as pool:
    results = pool.starmap(process_shard, zip(shards_images[:4], shards_labels[:4]))


Epoch 1/5
Epoch 1/5
Epoch 1/5
Epoch 1/5
3/3 [==============================] - 2s 310ms/step - loss: 7.7715 - accuracy: 0.3243
Epoch 2/5
3/3 [==============================] - 2s 438ms/step - loss: 8.0126 - accuracy: 0.2568
Epoch 2/5
3/3 [==============================] - 2s 554ms/step - loss: 3.7089 - accuracy: 0.4865
Epoch 3/5
3/3 [==============================] - 3s 512ms/step - loss: 7.4640 - accuracy: 0.3649
Epoch 2/5
3/3 [==============================] - 2s 496ms/step - loss: 13.9176 - accuracy: 0.3243
Epoch 3/5
3/3 [==============================] - 5s 504ms/step - loss: 10.7754 - accuracy: 0.3919
Epoch 4/5
Epoch 2/5
3/3 [==============================] - 2s 457ms/step - loss: 7.1307 - accuracy: 0.4459
Epoch 3/5
3/3 [==============================] - 2s 603ms/step - loss: 4.9938 - accuracy: 0.3649
Epoch 4/5
3/3 [==============================] - 2s 428ms/step - loss: 1.6610 - accuracy: 0.4865
Epoch 5/5
Epoch 3/5
3/3 [==============================] - 2s 447ms/step - loss: 4.72